# 📊 **Antigranular** Heart Disease Prediction Contest (ft. **Harvard/OpenDP** and **TPDP**)

🎉 Welcome to a new [Antigranular](https://antigranular.com) contest in collaboration with the [TPDP Workshop](https://tpdp.journalprivacyconfidentiality.org/2024/) and [Harvard's OpenDP Community Meeting](https://opendp.org/)!

🩺 This time, we are focusing on [heart condition detection](https://en.wikipedia.org/wiki/Cardiovascular_disease) using our new [TensorFlow Privacy](https://github.com/tensorflow/privacy) and [Opacus (PyTorch)](https://opacus.ai/) models!

🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!



## 🏃‍♂️ Getting Started

In this section we will download the antigranular package and login




### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [ ]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to [Competitions](https://www.antigranular.com/competitions) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [ ]:
import antigranular as ag


Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 110a95ed-a494-4256-adb2-045cf32208a4, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


### 🤖 Using AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [ ]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

### 🕵️‍♂️ Exploring data

Exploring data in Antigranular involves spending your epsilon budget, be mindful of your usage but remember that the less epsilon you use, the less accurate your results will get!

In [ ]:
# %%ag
# x_train.info()

In [ ]:
# %%ag
# y_train.info()

In [ ]:
# %%ag
# # We can start by exploring the data, carefully using our epsilon
# describe = x_train.describe(eps=0.1)
# ag_print(describe)

In [ ]:
# %%ag
# # We can start by exploring the data, carefully using our epsilon
# describe = y_train.describe(eps=0.1)
# ag_print(describe)

In [ ]:
%%ag
# x_test is a public test set, so we can print it without using epsilon
# ag_print(x_test)

### 🎈 A quick solution

In this section we evaluate an editorial solution in AG using TensorFlow!

In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader


# Normal keras model
seqM = Sequential([
    Dense(128, activation='relu', input_shape=(6,)),
    Dropout(0.05),
    Dense(256, activation='relu'),
    Dropout(0.05),
    Dense(128, activation='relu'),
    Dropout(0.05),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

In [ ]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=1)
x_train_scaled.info()

+----+----------+-------------+---------------+---------+------------------------------------------+
|    | Column   | numerical   | categorical   | dtype   | bounds                                   |
|----+----------+-------------+---------------+---------+------------------------------------------|
|  0 | age      | True        | False         | float64 | (-3.2251650958889058,                    |
|    |          |             |               |         | 3.0125396666840714)                      |
|  1 | sex      | True        | False         | float64 | (-1.435872236274705, 0.7004278340922758) |
|  2 | bp       | True        | False         | float64 | (-2.8170087590658563, 4.539453513068863) |
|  3 | ch       | True        | False         | float64 | (-2.507452232531863, 6.0456372000583976) |
|  4 | bs       | True        | False         | float64 | (-2.6891593723446596, 4.046803542053309) |
|  5 | phr      | True        | False         | float64 | (-4.007087401809214, 3.4166096792

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=2)

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=15, target_delta=1e-5)

Epoch 1/15

4000/4000 - 81s - loss: 0.4642 - accuracy: 0.7698 - 81s/epoch - 20ms/step

Epoch 2/15

4000/4000 - 76s - loss: 0.3170 - accuracy: 0.8539 - 76s/epoch - 19ms/step

Epoch 3/15

4000/4000 - 79s - loss: 0.2436 - accuracy: 0.8939 - 79s/epoch - 20ms/step

Epoch 4/15

4000/4000 - 79s - loss: 0.1980 - accuracy: 0.9180 - 79s/epoch - 20ms/step

Epoch 5/15

4000/4000 - 74s - loss: 0.1690 - accuracy: 0.9313 - 74s/epoch - 19ms/step

Epoch 6/15

4000/4000 - 75s - loss: 0.1435 - accuracy: 0.9441 - 75s/epoch - 19ms/step

Epoch 7/15

4000/4000 - 80s - loss: 0.1358 - accuracy: 0.9482 - 80s/epoch - 20ms/step

Epoch 8/15

4000/4000 - 75s - loss: 0.1211 - accuracy: 0.9552 - 75s/epoch - 19ms/step

Epoch 9/15



In [ ]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=1)
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 1s 6ms/step



In [ ]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

### 📝 Make your submission (Through AG)

Submit a prediction by simply typing `submit_predictions(your_prediction)` to find out how you rank on the leaderboard.

![img](https://www.antigranular.com/static/media/Step%209.8091828f3cff4324fe6d.png)


### 🎈 Another quick solution

In this section we evaluate an editorial solution in AG using Diffprivlib!

### 🎉 That's it!

Congrats! You made your first submission to the competition! Now it's time to keep exploring the data and to try to achieve a better score! Here are the next steps:


1.  🏫 Head over to our [Docs](https://docs.antigranular.com/) and discover Opacus (PyTorch), TensorFlow Privacy and other libraries available in Antigranular!
2. 🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!

We hope you have fun and enjoy the competition!

Best of luck,

Antigranular Team